In [395]:
import warnings
warnings.filterwarnings("ignore")

In [455]:
import pandas as pd
df = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/train_data.csv')
df.head()

,id,message
0,271828,Over $616 million in Bitcoin was electrocated ...
1,271829,Quiz: Thursday or friday?
2,271830,The Australian Revenue Authority will start co...
3,271831,Let's continue😉. I present to you my new review
4,271832,Here comes your future palette.


In [456]:
solution = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/train_solution.csv')
solution.head()

,id,category
0,271828,1
1,271829,0
2,271830,1
3,271831,2
4,271832,2


In [424]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Пробовала все кроме предобученных токенизаторов, в итоге лучше всего работает просто убрать пунктуацию и разделить сплитом (смайлики оставила, без них тоже пробовала)

In [457]:
from nltk.corpus import stopwords
import re
import nltk 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

def preprocess(text, remove_stopwords=True):
    text = re.sub("[\,\.\-\;\:\"?\']"," ", text)
    #words = word_tokenize(text) - еще хуже работает
    words = text.split()
    words = [word.lower() for word in words]
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

corpus = []
rows = []
for text in df["message"]:
    clean_text = preprocess(text)
    corpus += clean_text
    rows.append(clean_text)
df['message'] = rows
df.head(20)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ksushazinoveva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ksushazinoveva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,message
0,271828,"[$616, million, bitcoin, electrocated, septemb..."
1,271829,"[quiz, thursday, friday]"
2,271830,"[australian, revenue, authority, start, collec..."
3,271831,"[let, continue😉, present, new, review]"
4,271832,"[comes, future, palette]"
5,271833,"[next, lecture, class(18, april), prof, vasila..."
6,271834,"[post, whose, gilpilege, ukrainian, tv, show, ..."
7,271835,"[amazon, web, services, platform, alchemy, att..."
8,271836,"[continuing, retinol, festival, gymnasium, rea..."
9,271837,"[wine, gonna, talk, belishko, see, much, oxymi..."


Попробовала fastText, выбился неплхой скор, с ним не эксперементировала, так как он возможно (?) предобучен.

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3166311 sha256=530877566eeca3e7e50d9cf87dfe5bfb936a14d956bff5f76879b4e257d8b863
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [12]:
import fasttext

In [ ]:
df['target'] = solution['category'].apply(lambda x: '__label__' + str(x))

for i, s in enumerate(df['message']):
    df.loc[i, 'sentence'] = ' '.join(s)
df

,id,message,target,sentence
0,271828,"[$616, million, bitcoin, electrocated, septemb...",__label__1,$616 million bitcoin electrocated september wr...
1,271829,"[quiz:, thursday, friday?]",__label__0,quiz: thursday friday?
2,271830,"[australian, revenue, authority, start, collec...",__label__1,australian revenue authority start collecting ...
3,271831,"[let's, continue😉., present, new, review]",__label__2,let's continue😉. present new review
4,271832,"[comes, future, palette.]",__label__2,comes future palette.
...,...,...,...,...
3839,275667,"[air,, shooter's, kms, man, asked, favourite, ...",__label__2,"air, shooter's kms man asked favourite subpoen..."
3840,275668,"[shit, that's, off-the-books.]",__label__0,shit that's off-the-books.
3841,275669,"[time, go, sky,, tabor.]",__label__2,"time go sky, tabor."
3842,275670,[uraaaaaaaa],__label__0,uraaaaaaaa


In [ ]:
#df_train, df_test = train_test_split(df, test_size = 0.2, random_state=42, stratify = df['target'])
df_train, df_dev = train_test_split(df, test_size = 0.2, random_state=42, stratify = df['target'])

df_train.reset_index(drop=True, inplace=True)
#df_test.reset_index(drop=True, inplace=True)
df_dev.reset_index(drop=True, inplace=True)

df_train[['sentence', 'target']].to_csv('train.txt', header=False, index=False, sep="\t")

In [ ]:
ft_model = fasttext.train_supervised('train.txt', epoch=20)

#y_test = df_test['target'].to_list()
#y_pred_test = ft_model.predict(df_test['sentence'].to_list())[0]
#print(' test:', accuracy_score(y_test, y_pred_test))

y_train = df_train['target'].to_list()
y_pred_train = ft_model.predict(df_train['sentence'].to_list())[0]
print(' train:', accuracy_score(y_train, y_pred_train))

y_dev = df_dev['target'].to_list()
y_pred_dev = ft_model.predict(df_dev['sentence'].to_list())[0]
print(' dev:', accuracy_score(y_dev, y_pred_dev))

 train: 0.9983739837398374
 dev: 0.8790637191157347


In [ ]:
df_test = pd.read_csv('test_data.csv')
corpus = []
rows = []
for text in df_test["message"]:
    clean_text = preprocess(text)
    corpus += clean_text
    rows.append(clean_text)
df_test['message'] = rows
for i, s in enumerate(df_test['message']):
    df_test.loc[i, 'sentence'] = ' '.join(s)
df_test.head()

,id,message,sentence
0,275672,"[lot, people, job, fair, tonight.]",lot people job fair tonight.
1,275673,"[also,, got, 4, answers, google, form, ...]","also, got 4 answers google form ..."
2,275674,"[vladimir,, seminar?]","vladimir, seminar?"
3,275675,"[couple, 111,, too?]","couple 111, too?"
4,275676,"[anti-buying., again,, zara:]","anti-buying. again, zara:"


In [ ]:
#y_test_pred = ft_model.predict(df_test['sentence'].to_list())[0]
#y_test_pred 

In [ ]:
df_test['category'] = y_test_pred 

In [ ]:
df_test['category'] = df_test['category'].apply(lambda x: x[0][-1])
df_test.drop(columns=['message', 'sentence'], inplace = True)
df_test

,id,category
0,275672,2
1,275673,0
2,275674,0
3,275675,0
4,275676,0
...,...,...
5922,3143750,0
5923,3143751,1
5924,3143752,0
5925,3143753,2


In [ ]:
df_test.to_csv('test_submission.csv')

Попробовала CNN (вдохновлялась https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb и ДЗ по sentiment analysis с другого предмета)
      

In [458]:
df['category'] = solution['category']
df

,id,message,category
0,271828,"[$616, million, bitcoin, electrocated, septemb...",1
1,271829,"[quiz, thursday, friday]",0
2,271830,"[australian, revenue, authority, start, collec...",1
3,271831,"[let, continue😉, present, new, review]",2
4,271832,"[comes, future, palette]",2
...,...,...,...
3839,275667,"[air, shooter, kms, man, asked, favourite, sub...",2
3840,275668,"[shit, books]",0
3841,275669,"[time, go, sky, tabor]",2
3842,275670,[uraaaaaaaa],0


In [33]:
!pip3 install -U --force-reinstall gensim

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached gensim-4.2.0-cp39-cp39-macosx_10_9_x86_64.whl (24.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 2.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.2.1
    Uninstalling smart-open-5.2.1:
      Successfully uninstalled smart-open-5.2.1
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homeb

In [459]:
from gensim.corpora.dictionary import Dictionary

common_dictionary = Dictionary(df['message'])
d = common_dictionary.token2id

In [460]:
def map_words_to_ind(text, d):
    result = []
    for word in text:
        result.append(d[word])
    return result

df['message'] = df['message'].apply(lambda x: map_words_to_ind(x, d))

In [461]:
df['category'] = solution['category']

In [462]:
import numpy as np
padded_texts = []
max_len = 400

for row in df['message']:
    if len(row) < max_len:
        padded_texts.append(np.pad(row, (0, max_len-len(row)), 'constant', constant_values=(1)))
    else:
        arr = np.array(row)
        padded_texts.append(arr[:max_len])
    
df['padded'] = padded_texts

Изначально пользовалась валидацией, но с тестом они не сильно похожи получались + больше данных в трейне -> лучше обучается модель (судя по всему, раз предлагали скачивать другие данные).

In [463]:
#df_train, df_dev = train_test_split(df ,test_size = 0.1, random_state=42, stratify = df['category'])
df_train = df
df_train.reset_index(inplace=True, drop=True)
#df_dev.reset_index(inplace=True, drop=True)

In [464]:
import torch
# import torchtext 
# from torchtext import data
# from torchtext import datasets
import random
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
train_ds = Dataset(df_train.padded, df_train.category)
#dev_ds = Dataset(df_dev.padded, df_dev.category)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
#dev_dl = DataLoader(dev_ds, batch_size=128, shuffle=False)

In [465]:
from torch import nn
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        return self.fc(cat)

In [466]:
INPUT_DIM = len(d) + 1
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5, 3]
OUTPUT_DIM = 3
DROPOUT = 0.4

model_cnn = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, 1)

Тут был ДОЛГИЙ перебор параметров, оставила лучший вариант (на 0.8)

In [469]:
import torch.optim as optim

device = torch.device('cpu')
optimizer = optim.Adam(model_cnn.parameters(), lr=0.001)
#optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
model_cnn = model_cnn.to(device)
criterion = criterion.to(device)

In [470]:
from sklearn.metrics import f1_score, accuracy_score
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        texts = torch.tensor(batch[0], device=device).long()
        labels = torch.tensor(batch[1], device=device).long()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        acc = accuracy_score(labels.cpu().detach().numpy(), (predictions.argmax(dim=-1)).cpu().detach().numpy())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
  
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            texts = torch.tensor(batch[0], device=device).long()
            labels = torch.tensor(batch[1], device=device).long()
            predictions = model(texts)
            loss = criterion(predictions, labels)
            acc = accuracy_score(labels.cpu().detach().numpy(), (predictions.argmax(dim=-1)).cpu().detach().numpy())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [472]:
#0.8
N_EPOCHS = 15

best_valid_loss = float('inf')

train_losses = []
dev_losses = []

train_f1 = []
dev_f1 = []

for epoch in range(N_EPOCHS):
    print('Epoch:', epoch)
    train_loss, train_score = train(model_cnn, train_dl, optimizer, criterion)
    #dev_loss, dev_f1_score = evaluate(model_cnn, dev_dl, criterion)
    
    train_losses.append(train_loss)
    #dev_losses.append(dev_loss)
    
    train_f1.append(train_f1_score)
    #dev_f1.append(dev_f1_score)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_score:.2f}')
    #print(f'\t Dev. Loss: {dev_loss:.3f} |  Dev. F1: {dev_f1_score:.2f}')

Epoch: 0
	Train Loss: 0.851 | Train accuracy: 0.59
Epoch: 1
	Train Loss: 0.384 | Train accuracy: 0.90
Epoch: 2
	Train Loss: 0.231 | Train accuracy: 0.95
Epoch: 3
	Train Loss: 0.143 | Train accuracy: 0.97
Epoch: 4
	Train Loss: 0.095 | Train accuracy: 0.98
Epoch: 5
	Train Loss: 0.066 | Train accuracy: 0.99
Epoch: 6
	Train Loss: 0.052 | Train accuracy: 0.99
Epoch: 7
	Train Loss: 0.046 | Train accuracy: 0.99
Epoch: 8
	Train Loss: 0.037 | Train accuracy: 0.99
Epoch: 9
	Train Loss: 0.028 | Train accuracy: 1.00
Epoch: 10
	Train Loss: 0.024 | Train accuracy: 1.00
Epoch: 11
	Train Loss: 0.022 | Train accuracy: 1.00
Epoch: 12
	Train Loss: 0.021 | Train accuracy: 1.00
Epoch: 13
	Train Loss: 0.019 | Train accuracy: 1.00
Epoch: 14
	Train Loss: 0.022 | Train accuracy: 1.00


Обработаем тест и посчитаем предикты

In [473]:
from gensim.corpora.dictionary import Dictionary
padded_texts = []
max_len = 400

def map_words_to_ind_test(text, d):
    result = []
    for word in text:
        if word in d:
          result.append(d[word])
    return result

df_test = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/test_data.csv')
corpus = []
rows = []
for text in df_test["message"]:
    clean_text = preprocess(text)
    corpus += clean_text
    rows.append(clean_text)
df_test['message'] = rows

common_dictionary = Dictionary(df_test['message'])
d_test = common_dictionary.token2id

df_test['message'] = df_test['message'].apply(lambda x: map_words_to_ind_test(x, d))

for row in df_test['message']:
    if len(row) < max_len:
        padded_texts.append(np.pad(row, (0, max_len-len(row)), 'constant', constant_values=(1)))
    else:
        arr = np.array(row)
        padded_texts.append(arr[:max_len])
    
df_test['padded'] = padded_texts
df_test

,id,message,padded
0,275672,"[477, 168, 2422, 2625, 2201]","[477, 168, 2422, 2625, 2201, 1, 1, 1, 1, 1, 1,..."
1,275673,"[16, 648, 1458, 11320, 1373, 1513]","[16, 648, 1458, 11320, 1373, 1513, 1, 1, 1, 1,..."
2,275674,"[11203, 1448]","[11203, 1448, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,275675,"[1016, 15143]","[1016, 15143, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,275676,"[4474, 2473, 4514]","[4474, 2473, 4514, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
5922,3143750,"[313, 507, 1075, 1102, 3471, 755, 15774]","[313, 507, 1075, 1102, 3471, 755, 15774, 1, 1,..."
5923,3143751,"[5291, 3216, 1143, 22, 1938, 13863, 824]","[5291, 3216, 1143, 22, 1938, 13863, 824, 1, 1,..."
5924,3143752,"[8850, 6700, 1389, 1107, 3353, 56, 513, 1381]","[8850, 6700, 1389, 1107, 3353, 56, 513, 1381, ..."
5925,3143753,"[7067, 8774, 743, 394, 7039, 16740, 4015, 1160...","[7067, 8774, 743, 394, 7039, 16740, 4015, 1160..."


In [474]:
test_preds = model_cnn(torch.tensor(df_test['padded'], device=device).long())

In [475]:
#test_preds = test_preds.detach().numpy()
preds = test_preds.argmax(dim=-1).detach().numpy()

In [476]:
df_test = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/test_data.csv')
df_test['category'] = preds

Посмотрим на часто встречающиеся слова в трейне, если они специфические (например, для крипты таких слов много, как мне кажется), то сделаем доп костыли. Также пробовала несколько вариантов костылей, оставила лучшее. 

In [483]:
df = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/train_data.csv')

In [484]:
df['category'] = solution['category']

In [485]:
from collections import Counter
corpus = []
for text in df.loc[df['category'] == 0]["message"]:
    clean_text = preprocess(text)
    corpus += clean_text

freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
freq_dict_sorted[:20]

[('hey', 186),
 ('whoa', 76),
 (')', 70),
 ('uh', 68),
 ('lose', 61),
 ('gonna', 59),
 ('time', 53),
 ('one', 50),
 ('week', 47),
 ('like', 43),
 ('2', 42),
 ('data', 40),
 ('think', 40),
 ('got', 39),
 ('please', 36),
 ('course', 35),
 ('tomorrow', 34),
 ('get', 34),
 ('write', 34),
 ('need', 33)]

In [185]:
from collections import Counter
corpus = []
for text in df.loc[df['category'] == 1]["message"]:
    clean_text = preprocess(text)
    corpus += clean_text

freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
freq_dict_sorted[:20]

[('bitcoin', 828),
 ('million', 478),
 ('new', 464),
 ('market', 445),
 ('according', 443),
 ('exchange', 418),
 ('digital', 417),
 ('company', 397),
 ('also', 352),
 ('users', 302),
 ('000', 299),
 ('assets', 295),
 ('per', 290),
 ('time', 287),
 ('first', 275),
 ('network', 272),
 ('billion', 261),
 ('platform', 261),
 ('financial', 257),
 ('$', 251)]

In [188]:
from collections import Counter
corpus = []
for text in df.loc[df['category'] == 2]["message"]:
    clean_text = preprocess(text)
    corpus += clean_text

freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
freq_dict_sorted[:30]

[('hey', 935),
 ('like', 664),
 ("i'm", 656),
 ('gonna', 443),
 ("i've", 280),
 ('okay', 273),
 ("there's", 239),
 ('one', 232),
 ('know', 230),
 ('face', 218),
 ('time', 216),
 ('skin', 204),
 ('want', 196),
 ("that's", 191),
 ('good', 186),
 ('get', 185),
 ('little', 162),
 ('got', 158),
 ('day', 156),
 ('first', 155),
 ('think', 154),
 ('go', 149),
 ('rubles', 145),
 ('yeah', 142),
 ('new', 141),
 ('put', 141),
 ('oh', 141),
 ('take', 130),
 ('make', 128),
 ('even', 127)]

In [480]:
df_test

,id,message,category
0,275672,But a lot of people have a job fair tonight.,0
1,275673,"Also, I got only 4 answers on the google form ...",0
2,275674,"Vladimir, when will we have seminar?",0
3,275675,"A couple at 111, too?",0
4,275676,"It's on Anti-buying. And again, Zara:",2
...,...,...,...
5922,3143750,"The second thing, write all questions and thou...",0
5923,3143751,Can Grayscale Actually Sell Bitcoin (BTC) Afte...,1
5924,3143752,It's not a very operational dude putting them ...,0
5925,3143753,Inspired by Sonia Miro's goddess and made a he...,2


In [481]:
def special_words(x):
    if ('crypt' in x) or ('coin' in x) or ('blockchain' in x) or ('token' in x) or ('income' in x) or ('mining' in x) or ('financial' in x):
        return 1
    elif ('zara' in x) or ('sale' in x) or ('catalog' in x) or ('beauty' in x) or ('skin' in x):
        return 2
    elif ('quiz' in x) or ('seminar' in x):
        return 0
    else:
        return -1

In [482]:
df_test['message'] = df_test['message'].apply(lambda x: x.lower())

In [486]:
df_test['костыль'] = df_test['message'].apply(special_words)

In [487]:
indices = df_test.loc[(df_test['костыль'] != -1) & (df_test['костыль'] != df_test['category'])].index.to_list()

In [488]:
for i in indices:
    tmp =  df_test.iloc[i]['костыль']
    df_test.loc[i,'category'] = tmp

In [489]:
df_test.loc[(df_test['костыль'] != -1) & (df_test['костыль'] != df_test['category'])]

,id,message,category,костыль


In [490]:
df_test.drop(columns=['message', 'костыль'], inplace = True)

In [491]:
df_test.to_csv('/Users/ksushazinoveva/Downloads/public_data (1)/solution.csv')

Для задания с 'my future' просто обучу word2vec (без stopwords, так как тогда удаляется слово my). Хочу заметить, что этим векторным пространством я не пользовалась, но в задании не было на это ограничений. Удобнее использовать w2v, так как там есть функция most similar для нескольких слов. Конечно, можно было посчитать tf-idf и усреднить векторы слов 'my' и 'future' (либо посчитать их веса как веса idf), но w2v быстрее..

In [384]:
def calc_idf(texts):
    idf = dict()
    cnt = defaultdict(int)
    for doc in texts:
        tmp = set(doc)
        for elem in tmp:
            cnt[elem] += 1
    for key, value in cnt.items():
        idf[key] = log10(len(texts) / value)
    return idf

def calc_mean_idf(text, model, vec_size = None, idf = None):
    res = np.array([0.0] * vec_size)
    for word in text:
        if word in model.wv:
            res += model.wv[word] * idf[word]
    return res

In [385]:
df = pd.read_csv('/Users/ksushazinoveva/Downloads/public_data (1)/train_data.csv')

rows = []
for text in df["message"]:
    clean_text = preprocess(text, remove_stopwords=False)
    corpus += clean_text
    rows.append(clean_text)
df['message'] = rows
df.head(20)

,id,message
0,271828,"[over, $616, million, in, bitcoin, was, electr..."
1,271829,"[quiz, thursday, or, friday]"
2,271830,"[the, australian, revenue, authority, will, st..."
3,271831,"[let, s, continue😉, i, present, to, you, my, n..."
4,271832,"[here, comes, your, future, palette]"
5,271833,"[in, next, lecture, class(18, april), prof, va..."
6,271834,"[a, post, for, those, whose, gilpilege, is, a,..."
7,271835,"[the, amazon, web, services, platform, for, al..."
8,271836,"[we, re, continuing, the, retinol, festival, i..."
9,271837,"[i, ve, had, some, wine, so, we, re, gonna, ta..."


In [393]:
from gensim.models import word2vec
model = word2vec.Word2Vec(rows, workers=4, size=300, min_count=10, window=10, sample=1e-3)

In [394]:
print(model.wv.most_similar(['my', 'future']))

[('problem', 0.9979813098907471), ('choice', 0.9977887868881226), ('point', 0.9974707365036011), ('few', 0.9974046349525452), ('sells', 0.9967643022537231), ('started', 0.9963586330413818), ('class', 0.9962261319160461), ('stopped', 0.9961521029472351), ('news', 0.9960910081863403), ('thoughts', 0.9959324598312378)]


Попробовала поэкспериментировать с kmax-pooling-ом, но результата лучше,чем на сетке до этого не вышло. Реализация K_max_pooling взята [отсюда](https://discuss.pytorch.org/t/resolved-how-to-implement-k-max-pooling-for-cnn-text-classification/931/5)

In [435]:
def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)
  
class CNN1d_kmax(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_dim, dropout, kmax = 3):
        super(CNN1d_kmax, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.kmax = kmax
    
        self.conv1 = nn.Conv1d(embedding_dim, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=1)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(128 * kmax, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)
        
        conved = F.relu(self.conv1(embedded))
        pooled = F.max_pool1d(conved, 2)
        conved = F.relu(self.conv2(pooled))
        conved = F.relu(self.conv3(conved))
        pooled = kmax_pooling(conved, 2, 3)
        pooled = pooled.view(-1, 128 * self.kmax)
        drop = self.dropout(pooled) 
        
        logit = self.fc(drop)
        
        return F.log_softmax(logit, dim=1)

In [436]:
INPUT_DIM = len(d) + 1
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUTPUT_DIM = 3
DROPOUT = 0.5

model_cnn = CNN1d_kmax(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, DROPOUT)

In [437]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model_cnn.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()
model_cnn = model_cnn.to(device)
criterion = criterion.to(device)

In [438]:
N_EPOCHS = 15

best_valid_loss = float('inf')

train_losses = []
dev_losses = []

train_f1 = []
dev_f1 = []

for epoch in range(N_EPOCHS):
    print('Epoch:', epoch)
    train_loss, train_score = train(model_cnn, train_dl, optimizer, criterion)
    #dev_loss, dev_f1_score = evaluate(model_cnn, dev_dl, criterion)
    
    train_losses.append(train_loss)
    #dev_losses.append(dev_loss)
    
    train_f1.append(train_f1_score)
    #dev_f1.append(dev_f1_score)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_score:.2f}')
    #print(f'\t Dev. Loss: {dev_loss:.3f} |  Dev. F1: {dev_f1_score:.2f}')

Epoch: 0
	Train Loss: 0.915 | Train accuracy: 0.60
Epoch: 1
	Train Loss: 0.758 | Train accuracy: 0.66
Epoch: 2
	Train Loss: 0.658 | Train accuracy: 0.73
Epoch: 3
	Train Loss: 0.540 | Train accuracy: 0.79
Epoch: 4
	Train Loss: 0.460 | Train accuracy: 0.82
Epoch: 5
	Train Loss: 0.385 | Train accuracy: 0.86
Epoch: 6
	Train Loss: 0.353 | Train accuracy: 0.87
Epoch: 7
	Train Loss: 0.310 | Train accuracy: 0.89
Epoch: 8
	Train Loss: 0.264 | Train accuracy: 0.91
Epoch: 9
	Train Loss: 0.234 | Train accuracy: 0.92
Epoch: 10
	Train Loss: 0.198 | Train accuracy: 0.94
Epoch: 11
	Train Loss: 0.175 | Train accuracy: 0.94
Epoch: 12
	Train Loss: 0.152 | Train accuracy: 0.95
Epoch: 13
	Train Loss: 0.132 | Train accuracy: 0.96
Epoch: 14
	Train Loss: 0.170 | Train accuracy: 0.95


## Итог 
Пробить weak baseline так и не удалось, в основном все эксперименты ухудшали скор. В итоге комбинация параметров для сетки без макс пулинга выбивает 0.8, но это лучший результат. Думаю, что если бы я потратила больше времени на это задание и скачала бы больше данных (но тут вопрос с легетимностью использования гугл переводчика), то возможно удалось бы набрать чуть больший результат. Возможно, архитектура сети CNN - не самая подходящая для этой задачи, но тогда остается трансформер? 